In [ ]:
!pip install flair==0.8.0.post1

In [ ]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import re
import unicodedata
from unidecode import unidecode
from statistics import *

import json

import torch

import flair
from flair.data import Sentence, Dictionary, Corpus
from flair.models import SequenceTagger
from flair.datasets import ColumnCorpus, UD_ENGLISH
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus
from flair.trainers import ModelTrainer
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings,CharacterEmbeddings, FlairEmbeddings,BytePairEmbeddings



import spacy


<h3> BiLSTM+CRF </h3>

In [ ]:
# TAG TO PREDICT 
tag_type = 'ner'

# define columns
columns = {0 : 'text', 1 : 'ner'}

# initializing the corpus
corpus: Corpus = ColumnCorpus('../data/BIOES-scheme/10fold-cv',columns,
                              train_file = 'TRAIN/TRAIN_0.txt',
                              test_file = 'TEST/TEST_0.txt')
    
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# EMBEDDINGS
embedding_types = [
    
    WordEmbeddings('../PretrainedEmbedding/w2v/word2vec.model'),
    FlairEmbeddings('../PretrainedEmbedding/forward_flair/best-lm.pt'),
    FlairEmbeddings('../PretrainedEmbedding/backward_flair/best-lm.pt'),
    TransformerWordEmbeddings('camembert-base')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)



# INITIALISATION SEQUENCE TAGGER

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        dropout = 0.2,
                                        word_dropout = 0.05,
                                        use_crf=True,
                                        use_rnn=True)

# INITIALISATION TRAINER

trainer: ModelTrainer = ModelTrainer(tagger, corpus) #optimizer=torch.optim.Adam

# START TRAINING + SAUVEGARDE
trainer.train('../working/model/',
              learning_rate=0.1,
              mini_batch_size=32,
              embeddings_storage_mode='gpu',
              max_epochs=30)